In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from io import StringIO
from datetime import timedelta

#prevents warnings from being printed
import warnings
warnings.filterwarnings('ignore')

In [2]:
sd = pd.read_csv('sd_requests.csv')
sd = sd.convert_dtypes()
sd.head()

,id,status,desc,date,depts,docs,poc,msgs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D..."
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ..."
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,<NA>,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re..."
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re..."


In [3]:
sd.shape

(28848, 8)

### Add state and city column 

In [4]:
sd = sd.rename(columns = {'date': 'request date'})
sd = sd.fillna('')

sd['city'] = 'San Diego'
sd['state'] = 'CA'

### Convert msgs and docs column to dataframe for info extraction

In [5]:
csv_to_df = lambda csv: pd.read_csv(StringIO(csv)) if csv else None
sd['docs_csv'] = sd['docs'].apply(csv_to_df)
sd['msgs_csv'] = sd['msgs'].apply(csv_to_df)

sd['docs_csv'] = sd['docs_csv'].apply(lambda df: df.convert_dtypes().fillna('') if df is not None else None)
sd['msgs_csv'] = sd['msgs_csv'].apply(lambda df: df.convert_dtypes().fillna('') if df is not None else None)

In [6]:
sd['num docs'] = sd['docs_csv'].map(lambda df: len(df) if df is not None else 0)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...","December 7, 2015 via web",Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,"December 7, 2015 via web",City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,"December 7, 2015 via web",Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...","December 7, 2015 via web",Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


### Convert request dates into datetime objects

In [7]:
sd['request date'] = sd['request date'].str.extract(r'([A-z][a-z]+\s+\d{1,2},\s+\d{4})')
sd['request date'] = pd.to_datetime(sd['request date'], infer_datetime_format=True)

In [8]:
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2


In [9]:
#number of open and closed requests for san diego
pd.value_counts(sd['status'])

CLOSED    28843
OPEN          5
Name: status, dtype: Int64

In [10]:
#pd.Series(sd['depts'].unique()).to_frame().to_csv('unique sd depts.csv')

### Check types of departments within dataframe

In [11]:
dept_count = sd.groupby('depts').count().reset_index()
dept_count = dept_count[['depts', 'id']].rename(columns = {'id': 'requests received'})
dept_count

,depts,requests received
0,,143
1,ADA Compliance & Accessibility,6
2,Airports,41
3,Animal Services,849
4,Assistant Chief Operating Officer,4
...,...,...
469,"Transportation, Performance & Analytics",6
470,"Transportation, Public Records Administration",21
471,"Transportation, Public Records Administration ...",1
472,"Transportation, Stormwater",1


### Not every message has a government code. We will display only the ones where govt code is not null

In [12]:
def get_code(msgs):
    a = []
    if re.findall("Government Code section (.*)", msgs) != a:
        return str(re.findall("Government Code section (.[0-9]*\(.\))", msgs)).strip("[").strip("]").strip('\'').strip("§")

In [13]:
sd["Gov Code"] = sd["msgs"].apply(get_code)

In [14]:
sd[sd["Gov Code"].notnull()]

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code
8,15-1818,CLOSED,Incident and supplemental reports for: August ...,2015-12-07,Public Records Administration,,Lea Fields-Bernard,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,6254(c)
30,15-1840,CLOSED,Writings and communications related to Sai Kir...,2015-12-09,Deputy Chief Operating Officer - Infrastructur...,"title,link Letter to Finch (CCPRA 2015-1840).p...",Jacqueline Palmer,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,title \ 0 Request ...,2,6254(k)
39,15-1849,CLOSED,Communications and or calendar appointments be...,2015-12-10,Public Records Administration,,Lea Fields-Bernard,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,None,title \ 0 Request ...,0,6254(a)
49,15-1859,CLOSED,All records related to the demographic study c...,2015-12-11,Police,,Humberto Hernandez,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,None,title \ 0 Request ...,0,6254(c)
55,15-1865,CLOSED,Records related to the stadium EIR including s...,2015-12-14,Public Records Administration,"title,link Copy of S16025 Labor Details 12-16-...",Lea Fields-Bernard,"title,item,time ""Request Closed Public"",Govern...",San Diego,CA,...,title \ 0 R...,2,6254(c)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28835,21-5530,CLOSED,"Dear Custodian of Records, Under the Californ...",2021-10-26,Police,,Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,None,title \ 0 Requ...,0,6254(f)
28839,21-5550,CLOSED,I request any police reports involving inciden...,2021-10-27,Police,,Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,None,title \ 0 Requ...,0,6254(k)
28843,21-5579,CLOSED,"Hello, I would like a copy of the report from ...",2021-10-28,Animal Services,,Lori Hernandez,"title,item,time ""Request Published Public"",,""N...",San Diego,CA,None,title \ 0 Requ...,0,6254(f)
28845,21-5584,CLOSED,request for call for service 2110020816,2021-10-28,Police,"title,link 2110020816_Redacted.pdf,https://san...",Lori Hernandez,"title,item,time ""Request Published Public"",,""N...",San Diego,CA,title ...,title \ 0 Requ...,1,6254(f)


### Pull out response from each message in a new column

In [15]:
def get_response(msgs):
    return re.findall("Public\",.*\".*\n.{5,}\.", msgs)

In [16]:
sd["Response"] = sd["msgs"].apply(get_response)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[]
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[]
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[]
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[]
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[]


### Get the Submission Time 

In [17]:
def get_submission_time(msgs):
    return str(re.findall("Request received via web,(.*[0-9]*20..,.*:{1}..[pa]{1}m{1})", msgs)).strip("[").strip(
        "]").strip("'").strip("\"")

In [18]:
sd["Submission Time (str)"] = sd["msgs"].apply(get_submission_time)

sd["Submission Time (dt)"] = pd.to_datetime(sd["Submission Time (str)"])

sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt)
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[],"December 7, 2015, 5:08pm",2015-12-07 17:08:00
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:22pm",2015-12-07 17:22:00
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[],"December 7, 2015, 5:29pm",2015-12-07 17:29:00
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:33pm",2015-12-07 17:33:00
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[],"December 7, 2015, 5:39pm",2015-12-07 17:39:00


### Getting the  Closing Times

In [19]:
def extract_time(tbl):
    d = {8: True, -1: False}
    closed_df = tbl[tbl['title'].str.find('Closed').map(d)]
    closing = closed_df['time'].str.extract(r'([A-z][a-z]+\s+\d{1,2},\s+\d{4}, ([0-1]?[0-9]|2[0-3]):[0-5][0-9][a|p]m)')
    return closing.iloc[0, 0]

In [20]:
sd['Closing Time (str)'] = sd['msgs_csv'].apply(extract_time)
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time (str)
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[],"December 7, 2015, 5:08pm",2015-12-07 17:08:00,"December 17, 2015, 4:34pm"
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:22pm",2015-12-07 17:22:00,"December 15, 2015, 7:12am"
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[],"December 7, 2015, 5:29pm",2015-12-07 17:29:00,"December 22, 2015, 11:32am"
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:33pm",2015-12-07 17:33:00,"December 15, 2015, 7:10am"
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[],"December 7, 2015, 5:39pm",2015-12-07 17:39:00,"December 24, 2015, 10:34am"


In [21]:
"""def get_closing_time(msgs):
    #messages = row["msgs"]
    times = re.findall("[A-Z].{,10}[0-9]{,3}, [0-9]{4}, [0-9]{,2}:[0-9]*[pa]m", msgs)
    doc_updates = re.findall("Department|Document... Released|Request [A-Z].*",msgs )
    i=0
    for string in doc_updates:
        if "Closed" in string:
            break
        else:
            i+=1
    return times[i]

sd["Closing Time"] = sd["msgs"].apply(get_closing_time)
sd.head()"""

'def get_closing_time(msgs):\n    #messages = row["msgs"]\n    times = re.findall("[A-Z].{,10}[0-9]{,3}, [0-9]{4}, [0-9]{,2}:[0-9]*[pa]m", msgs)\n    doc_updates = re.findall("Department|Document... Released|Request [A-Z].*",msgs )\n    i=0\n    for string in doc_updates:\n        if "Closed" in string:\n            break\n        else:\n            i+=1\n    return times[i]\n\nsd["Closing Time"] = sd["msgs"].apply(get_closing_time)\nsd.head()'

In [22]:
sd['Closing Time (dt)'] = pd.to_datetime(sd['Closing Time (str)'].astype(str))
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time (str),Closing Time (dt)
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[],"December 7, 2015, 5:08pm",2015-12-07 17:08:00,"December 17, 2015, 4:34pm",2015-12-17 16:34:00
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:22pm",2015-12-07 17:22:00,"December 15, 2015, 7:12am",2015-12-15 07:12:00
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[],"December 7, 2015, 5:29pm",2015-12-07 17:29:00,"December 22, 2015, 11:32am",2015-12-22 11:32:00
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:33pm",2015-12-07 17:33:00,"December 15, 2015, 7:10am",2015-12-15 07:10:00
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[],"December 7, 2015, 5:39pm",2015-12-07 17:39:00,"December 24, 2015, 10:34am",2015-12-24 10:34:00


### Compute average time it takes for SD police department to complete a request

In [23]:
sd['days_until_completed'] = sd['Closing Time (dt)'] - sd['Submission Time (dt)']
sd.head()

,id,status,desc,request date,depts,docs,poc,msgs,city,state,docs_csv,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time (str),Closing Time (dt),days_until_completed
0,15-1810,CLOSED,"Notices of Violation/Notice to Comply, fire in...",2015-12-07,Code Enforcement,"title,link 5040 ShorehamPlace building permits...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ ...,title ...,2,None,[],"December 7, 2015, 5:08pm",2015-12-07 17:08:00,"December 17, 2015, 4:34pm",2015-12-17 16:34:00,9 days 23:26:00
1,15-1811,CLOSED,The October 2015 monthly report for SeaWorld,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Published Public"",,""D...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:22pm",2015-12-07 17:22:00,"December 15, 2015, 7:12am",2015-12-15 07:12:00,7 days 13:50:00
2,15-1812,CLOSED,Records related to the following BIDS: Adams ...,2015-12-07,City Clerk,"title,link http://www.sandiego.gov/park-and-re...",Mailei Ross-Cerezo,"title,item,time ""Request Closed Public"",Still ...",San Diego,CA,...,title \ 0 R...,1,None,[],"December 7, 2015, 5:29pm",2015-12-07 17:29:00,"December 22, 2015, 11:32am",2015-12-22 11:32:00,14 days 18:03:00
3,15-1813,CLOSED,Historical lease payments made by SeaWorld to ...,2015-12-07,Department of Real Estate and Airport Management,,Jeffrey Wallace,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,None,title ...,0,None,[],"December 7, 2015, 5:33pm",2015-12-07 17:33:00,"December 15, 2015, 7:10am",2015-12-15 07:10:00,7 days 13:37:00
4,15-1814,CLOSED,"Open violations, variances, ordinances, approv...",2015-12-07,Code Enforcement,"title,link Site Plan - 11943 El Camino Real.pd...",Ginger Rodriguez,"title,item,time ""Request Closed Public"",02. Re...",San Diego,CA,title \ 0 ...,title ...,2,None,[],"December 7, 2015, 5:39pm",2015-12-07 17:39:00,"December 24, 2015, 10:34am",2015-12-24 10:34:00,16 days 16:55:00


### Calculate Mean Time for police and non-police requests

In [24]:
sd_police = sd[sd['depts'].str.contains("(?i)police|sheriff|public safety", case = False)]

In [25]:
sd_police.groupby('status').sum()

,num docs
status,
CLOSED,12331
OPEN,3


In [26]:
print("Mean time for SD police dept to complete a request: ", sd_police['days_until_completed'].mean().round('1min'))

Mean time for SD police dept to complete a request:  12 days 16:08:00


In [27]:
sd_nonpolice = sd[~sd['depts'].str.contains("(?i)police|sheriff|public safety", case = False)]

In [28]:
sd_nonpolice.groupby('status').sum()

,num docs
status,
CLOSED,65415
OPEN,28


In [29]:
print("Mean time for non SD police dept to complete a request: ", sd_nonpolice['days_until_completed'].mean().round('1min'))

Mean time for non SD police dept to complete a request:  21 days 04:36:00


### Find results

In [227]:
def get_result(x):
    x = x.split('\n')
    for i in range(len(x)):
        if 'Request Close' in x[i]:
            y= x[i+1]
            z = y.split(',')
            z = z[1].replace('"', "")
            if z == 'Request Re-opened after Initial Closure':
                continue
            else:
                break
                
    y = y.split(',')
    res = y[1].replace('"', "")
    
    if '6254' in res:
        x.sort()
        for i in x:
            if '02.' in i or '02a.' in i or '02b.' in i or '02c.' in i or '01a.' in i:
                res = i
                break
            elif 'Document(s) Released' in i:
                res = '02c.'
                break
                
    if '02.' in res or 'All responsive documents have been released pursuant to the California Public Records Act' in res:
        return 'Released'
    elif '02a.' in res:
        return 'Released - Redacted'
    elif '02c.' in res or 'public interest in the nondisclosure of' in res:
        return 'Released - Redacted/Withheld'
    elif 'been released except those that have been redacted and/or withheld' in res:
        return 'Released - Redacted/Withheld'
    elif '02b.' in res:
        return 'Released - Withheld'
    elif '01a.' in res:
        return 'Not Released'
    elif 'Waiting for' in res:
        return "Waiting for resquester's response"
    elif '6254' in res:
        return 'Not Released'
    elif 'no responsive' in res or 'does not maintain the records' in res or 'No responsive documents.':
        return 'No responsive documents'
    else:
        return res

In [228]:
sd['result'] = sd['msgs'].apply(get_result)

In [204]:
sd['result'].value_counts().to_csv('results.csv')

In [229]:
sd['result'].value_counts().head(30)

Released                                                                                                                     10602
Released - Redacted                                                                                                           4746
No responsive documents                                                                                                       4253
Duplicate request                                                                                                             1670
Released - Redacted/Withheld                                                                                                  1651
Referred to another agency                                                                                                    1354
Not Released                                                                                                                  1055
Records available for review                                                       

In [233]:
temp = sd[sd['result'] == 'Released - Redacted/Withheld']

In [234]:
temp

,id,status,desc,request date,depts,docs,poc,msgs,city,state,...,msgs_csv,num docs,Gov Code,Response,Submission Time (str),Submission Time (dt),Closing Time (str),Closing Time (dt),days_until_completed,result
7,15-1817,CLOSED,File materials related to: P12010041171 P12050...,2015-12-07,Police,,Humberto Hernandez,"title,item,time ""Request Closed Public"",02c. R...",San Diego,CA,...,title ...,0,None,[],"December 8, 2015, 11:40am",2015-12-08 11:40:00,"December 21, 2015, 10:48am",2015-12-21 10:48:00,12 days 23:08:00,Released - Redacted/Withheld
55,15-1865,CLOSED,Records related to the stadium EIR including s...,2015-12-14,Public Records Administration,"title,link Copy of S16025 Labor Details 12-16-...",Lea Fields-Bernard,"title,item,time ""Request Closed Public"",Govern...",San Diego,CA,...,title \ 0 R...,2,6254(c),[],"December 15, 2015, 11:13am",2015-12-15 11:13:00,"December 29, 2015, 1:56pm",2015-12-29 13:56:00,14 days 02:43:00,Released - Redacted/Withheld
80,15-1890,CLOSED,Latest emergency action plans for the followin...,2015-12-17,Public Utilities,,Wilson Kennedy,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,title \ 0 Request ...,0,None,[],"December 18, 2015, 10:13am",2015-12-18 10:13:00,"January 11, 2016, 7:52am",2016-01-11 07:52:00,23 days 21:39:00,Released - Redacted/Withheld
92,15-1902,CLOSED,"Bills, statements, invoices received by the ci...",2015-12-17,Public Records Administration,"title,link AECOM_103015.pdf,https://sandiego.n...",Lea Fields-Bernard,"title,item,time ""Request Closed Public"",Govern...",San Diego,CA,...,title ...,7,6254(c),[],"December 21, 2015, 4:48pm",2015-12-21 16:48:00,"December 28, 2015, 5:34pm",2015-12-28 17:34:00,7 days 00:46:00,Released - Redacted/Withheld
119,15-1929,CLOSED,"Records relating to particular law suits, agre...",2015-12-24,City Attorney,"title,link Public Records Request.msg,https://...",Nancy Shapiro,"title,item,time ""Request Closed Hide Public""...",San Diego,CA,...,title \ 0 Request ...,1,None,"[Public"",""Still reviewing potentially responsi...","December 24, 2015, 12:36pm",2015-12-24 12:36:00,"January 8, 2016, 9:45am",2016-01-08 09:45:00,14 days 21:09:00,Released - Redacted/Withheld
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28693,21-5301,CLOSED,report # 21206787 car accident - El Cajon blvd...,2021-10-11,Police,"title,link 21206787_Redacted.pdf,https://sandi...",Lori Hernandez,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,...,title \ 0 Requ...,1,"6254(f)', '6254(k)",[],"October 11, 2021, 9:34am",2021-10-11 09:34:00,"October 12, 2021, 9:19am",2021-10-12 09:19:00,0 days 23:45:00,Released - Redacted/Withheld
28699,21-5308,CLOSED,Attn: Records Division City of San Diego Re: ...,2021-10-11,Police,"title,link 21-5308 IDR FS21151121_Redacted.pdf...",Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,...,title \ 0 Requ...,3,"6254(f)', '6254(f)', '6254(f)","[Public"",""21-5308 IDR FS21151121_Redacted.pdf\...","October 11, 2021, 12:43pm",2021-10-11 12:43:00,"October 18, 2021, 12:51pm",2021-10-18 12:51:00,7 days 00:08:00,Released - Redacted/Withheld
28702,21-5311,CLOSED,"My son, Richard Heuser, died by suicide 9/15. ...",2021-10-11,Police,"title,link 21-5311 IDR FS21149031_Redacted.pdf...",Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,...,title \...,1,"6254(f)', '6254(f)', '6254(f)",[],"October 11, 2021, 1:47pm",2021-10-11 13:47:00,"October 18, 2021, 12:38pm",2021-10-18 12:38:00,6 days 22:51:00,Released - Redacted/Withheld
28735,21-5351,CLOSED,"I am requesting all files that pertain to I, S...",2021-10-13,Police,"title,link E20040042357_Redacted.pdf,https://s...",Angela Laurita,"title,item,time ""Request Published Public"",,""O...",San Diego,CA,...,title \ 0 Requ...,2,6254(f),"[Public"",""E20040042357_Redacted.pdf\nArrest Re...","October 13, 2021, 9:07pm",2021-10-13 21:07:00,"October 15, 2021, 7:20am",2021-10-15 07:20:00,1 days 

In [223]:
temp.iloc[2,7].split('\n')

['title,item,time',
 '"Request Closed',
 'Public","Media Inquiry',
 'This request was handled as a media inquiry by Communications staff.","March 27, 2016, 1:13pm by Lea Fields-Bernard, Public Records Administration Manager, PRA Program Manager"',
 '"Request Reopened',
 'Public",,"March 27, 2016, 1:13pm by Lea Fields-Bernard, Public Records Administration Manager, PRA Program Manager"',
 '"Request Published',
 'Public",,"March 20, 2016, 5:19pm"',
 '"Request Closed',
 'Public","Media Inquiry',
 'This request was handled as a media inquiry by Communications staff.","March 17, 2016, 5:19pm by Lea Fields-Bernard, Public Records Administration Manager, PRA Program Manager"',
 '"Department Assignment',
 'Public",Public Records Administration,"March 14, 2016, 10:32am by Lea Fields-Bernard, Public Records Administration Manager, PRA Program Manager"',
 '"Request Opened',
 'Public",Request received via web,"March 14, 2016, 10:32am by Lea Fields-Bernard, Public Records Administration Manager, PR

In [230]:
len(sd)

28848